# Scraping Paris.fr - Appels à Projets

Ce notebook scrape les appels à projets du site Paris.fr et les mappe au format `mapped_df`.

**Source :** https://www.paris.fr/appels-a-projets

## 1. Imports & Configuration

In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import json
from urllib.parse import urljoin
import time

## 2. Scraper les appels à projets de Paris.fr

In [ ]:
# Configuration
BASE_URL = "https://www.paris.fr/appels-a-projets"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# Récupérer le HTML de la première page
print(f"🔄 Fetching {BASE_URL}...")
try:
    response = requests.get(BASE_URL, headers=HEADERS, timeout=10)
    response.raise_for_status()
    html_content = response.text
    print(f"✅ Page récupérée ({len(html_content)} caractères)")
except Exception as e:
    print(f"❌ Erreur: {str(e)}")
    html_content = None

��� Fetching https://www.paris.fr/appels-a-projets...
✅ Page récupérée (312332 caractères)


In [3]:
# Parser le HTML
if html_content:
    soup = BeautifulSoup(html_content, 'html.parser')
    print(f"✅ HTML parsé avec BeautifulSoup")
    print(f"   Titre de la page: {soup.title.string if soup.title else 'N/A'}")
else:
    soup = None
    print("⚠️ Pas de contenu HTML à parser")

✅ HTML parsé avec BeautifulSoup
   Titre de la page: Appels à projets - Ville de Paris


In [ ]:
# Fonction pour extraire les données des appels à projets
def scrape_paris_aap(soup):
    """Scrape les AAP du site Paris.fr"""
    aap_list = []
    
    if not soup:
        return aap_list
    
    # Chercher les conteneurs des AAP (peuvent être articles, divs, liens, etc.)
    # On cherche d'abord des patterns courants
    article_containers = soup.find_all(['article', 'div'], class_=re.compile(r'(appel|project|call|aap)', re.I))
    
    if not article_containers:
        # Fallback: chercher tous les liens potentiels vers les AAP
        article_containers = soup.find_all('a', href=re.compile(r'appel|project', re.I))
    
    print(f"📝 Trouvé {len(article_containers)} conteneurs potentiels")
    
    for container in article_containers:
        try:
            aap = {}
            
            # Titre
            title_elem = container.find(['h2', 'h3', 'h4', 'a'])
            aap['titre'] = title_elem.get_text(strip=True) if title_elem else 'N/A'
            
            # URL
            if container.name == 'a':
                aap['url_source'] = urljoin(BASE_URL, container.get('href', ''))
            else:
                link = container.find('a', href=True)
                aap['url_source'] = urljoin(BASE_URL, link.get('href', '')) if link else None
            
            # Description/résumé
            desc = container.find(['p', 'span'], class_=re.compile(r'(desc|summary|excerpt)', re.I))
            aap['resume'] = desc.get_text(strip=True) if desc else None
            
            # Dates (chercher des patterns comme "01/01/2026")
            text_content = container.get_text(' ')
            dates = re.findall(r'\d{1,2}/\d{1,2}/\d{4}', text_content)
            if len(dates) >= 2:
                try:
                    aap['date_publication'] = pd.to_datetime(dates[0], format='%d/%m/%Y').date()
                    aap['date_limite'] = pd.to_datetime(dates[1], format='%d/%m/%Y').date()
                except:
                    aap['date_publication'] = None
                    aap['date_limite'] = None
            elif len(dates) == 1:
                try:
                    aap['date_limite'] = pd.to_datetime(dates[0], format='%d/%m/%Y').date()
                    aap['date_publication'] = None
                except:
                    aap['date_publication'] = None
                    aap['date_limite'] = None
            else:
                aap['date_publication'] = None
                aap['date_limite'] = None
            
            # Montant (chercher des patterns comme "€", "euros")
            amounts = re.findall(r'(\d+[\s.,]*\d*)\s*(?:€|euros?|EUROS?)', text_content, re.I)
            if amounts:
                try:
                    # Nettoyer et convertir
                    cleaned = amounts[-1].replace(' ', '').replace('.', '').replace(',', '.')
                    aap['montant_max'] = float(cleaned)
                except:
                    aap['montant_max'] = None
            else:
                aap['montant_max'] = None
            
            # Organisme (chercher des balises spécifiques)
            org = container.find(['span', 'p'], class_=re.compile(r'(organisme|provider|publisher)', re.I))
            aap['organisme'] = org.get_text(strip=True) if org else 'Mairie de Paris'
            
            # Catégories/thèmes
            categories_elem = container.find_all(['span', 'a'], class_=re.compile(r'(categor|tag|theme)', re.I))
            if categories_elem:
                aap['categories'] = [cat.get_text(strip=True) for cat in categories_elem]
            else:
                aap['categories'] = None
            
            # ID unique
            aap['id_record'] = f"paris_{datetime.now().strftime('%Y%m%d%H%M%S')}_{hash(aap['titre']) % 10000}"
            
            aap_list.append(aap)
            
        except Exception as e:
            print(f"  ⚠️ Erreur parsing: {str(e)}")
            continue
    
    return aap_list

# Scraper les AAP
aap_data = scrape_paris_aap(soup)
print(f"\n✅ {len(aap_data)} appels à projets extraits")

��� Trouvé 12 conteneurs potentiels

✅ 12 appels à projets extraits


## 3. Créer un DataFrame brut

In [ ]:
# Créer DataFrame
df_paris = pd.DataFrame(aap_data)
print(f"📊 DataFrame créé: {df_paris.shape}")
print(f"\n📋 Colonnes: {list(df_paris.columns)}")
print(f"\n📋 Aperçu des données:")
df_paris.head()

��� DataFrame créé: (12, 9)

��� Colonnes: ['titre', 'url_source', 'resume', 'date_publication', 'date_limite', 'montant_max', 'organisme', 'categories', 'id_record']

��� Aperçu des données:


,titre,url_source,resume,date_publication,date_limite,montant_max,organisme,categories,id_record
0,Les appels à projets de la Ville de Paris,https://www.paris.fr/pages/repondre-a-un-appel...,None,None,None,None,Mairie de Paris,None,paris_20260102130659_7133
1,Appel à projets pour le partage de la culture ...,https://www.paris.fr/pages/appel-a-projets-202...,None,2025-01-16,2025-03-10,None,Mairie de Paris,None,paris_20260102130659_7444
2,Appel à projets « Prévenir les conduites à ris...,https://www.paris.fr/pages/appel-a-projets-pre...,None,2025-12-16,2026-02-16,None,Mairie de Paris,None,paris_20260102130659_3733
3,Appel à projets « Paris Fabrik » : formations ...,https://www.paris.fr/pages/appel-a-projets-par...,None,2025-12-15,2026-02-19,None,Mairie de Paris,None,paris_20260102130659_6413
4,Appel à projets immobiliers 2026 pour des lieu...,https://www.paris.fr/pages/appel-a-projets-imm...,None,2026-12-10,2026-04-02,None,Mairie de Paris,None,paris_20260102130659_1238
5,Appel à propositions pour un emplacement comme...,https://www.paris.fr/pages/appel-a-proposition...,None,2025-12-05,2026-01-14,None,Mairie de Paris,None,paris_20260102130659_4100
6,Appel à projets « Parcours linguistiques à vis...,https://www.paris.fr/pages/appel-a-projets-par...,None,2025-12-04,2026-02-03,None,Mairie de Paris,None,paris_20260102130659_6389
7,Appel à projets : Actions de prévention de la ...,https://www.paris.fr/pages/appel-a-projets-act...,None,2025-11-21,2026-01-09,None,Mairie de Paris,None,paris_20260102130659_9592
8,Appel à projets des activités péri et extrasco...,https://www.paris.fr/pages/appel-a-projets-des...,None,2025-11-03,2026-01-05,None,Mairie de Paris,None,paris_20260102130659_6449
9,Appel à projets des temps d'activités périscol...,https://www.paris.fr/pages/appel-a-projets-des...,None,2025-11-03,2026-01-05,None,Mairie de Paris,None,paris_20260102130659_8825


## 4. Nettoyer et normaliser les données

In [6]:
# Fonction de nettoyage du texte
def clean_text(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r'<[^>]+>', '', text)  # Supprimer les tags HTML
    html_entities = {
        '&eacute;': 'é', '&icirc;': 'î', '&agrave;': 'à',
        '&nbsp;': ' ', '&quot;': '\"', '&amp;': '&',
        '&rsquo;': "'", '&ldquo;': '\u201c', '&rdquo;': '\u201d'
    }
    for entity, char in html_entities.items():
        text = text.replace(entity, char)
    text = re.sub(r'\s+', ' ', text)  # Normaliser les espaces
    return text.strip()

# Appliquer le nettoyage
for col in df_paris.select_dtypes(include=['object']).columns:
    if col not in ['categories']:
        df_paris[col] = df_paris[col].apply(clean_text)

print("✅ Texte nettoyé")

✅ Texte nettoyé


In [ ]:
# Extraire les montants maximums
def extract_max_amount(text):
    if not isinstance(text, str):
        return None
    
    text = text.replace('\xa0', ' ')  # Non-breaking space
    amounts = re.findall(r'\b(\d+[\s.,]*\d*)\s*(?:€|euros?|EUROS?)', text)
    
    if not amounts:
        return None
    
    cleaned_amounts = []
    for amt in amounts:
        amt = amt.replace(' ', '')
        if ',' in amt:
            amt = amt.replace('.', '').replace(',', '.')
        else:
            parts = amt.split('.')
            if len(parts) > 1 and len(parts[-1]) == 3 and parts[-1].isdigit():
                amt = amt.replace('.', '')
        try:
            cleaned_amounts.append(float(amt))
        except ValueError:
            continue
    
    return max(cleaned_amounts) if cleaned_amounts else None

# Appliquer sur resume si montant_max est vide
for idx, row in df_paris.iterrows():
    if pd.isna(row['montant_max']) and pd.notna(row['resume']):
        df_paris.at[idx, 'montant_max'] = extract_max_amount(row['resume'])

print("✅ Montants extraits")

✅ Montants extraits


## 5. Mapper au format mapped_df

In [ ]:
# Créer le DataFrame mappé
mapped_df_paris = pd.DataFrame()

# Mapping direct des colonnes
if 'id_record' in df_paris.columns:
    mapped_df_paris['id_record'] = df_paris['id_record']
if 'titre' in df_paris.columns:
    mapped_df_paris['titre'] = df_paris['titre']
if 'organisme' in df_paris.columns:
    mapped_df_paris['organisme'] = df_paris['organisme']
if 'date_publication' in df_paris.columns:
    mapped_df_paris['date_publication'] = df_paris['date_publication']
if 'date_limite' in df_paris.columns:
    mapped_df_paris['date_limite'] = df_paris['date_limite']
if 'categories' in df_paris.columns:
    mapped_df_paris['categories'] = df_paris['categories']
if 'montant_max' in df_paris.columns:
    mapped_df_paris['montant_max'] = df_paris['montant_max']
if 'url_source' in df_paris.columns:
    mapped_df_paris['url_source'] = df_paris['url_source']
if 'resume' in df_paris.columns:
    mapped_df_paris['resume'] = df_paris['resume']

# Ajouter des colonnes manquantes avec valeurs par défaut
if 'public_cible' not in mapped_df_paris.columns:
    mapped_df_paris['public_cible'] = None
if 'taux_financement' not in mapped_df_paris.columns:
    mapped_df_paris['taux_financement'] = None
if 'contact' not in mapped_df_paris.columns:
    mapped_df_paris['contact'] = None
if 'modalite' not in mapped_df_paris.columns:
    mapped_df_paris['modalite'] = None
if 'demarches' not in mapped_df_paris.columns:
    mapped_df_paris['demarches'] = None
if 'mots_cles' not in mapped_df_paris.columns:
    mapped_df_paris['mots_cles'] = None
if 'objectif' not in mapped_df_paris.columns:
    mapped_df_paris['objectif'] = None
if 'montant_min' not in mapped_df_paris.columns:
    mapped_df_paris['montant_min'] = None
if 'note' not in mapped_df_paris.columns:
    mapped_df_paris['note'] = None
if 'tags' not in mapped_df_paris.columns:
    mapped_df_paris['tags'] = None
if 'perimetre_geo' not in mapped_df_paris.columns:
    mapped_df_paris['perimetre_geo'] = 'Paris'

print(f"✅ DataFrame mappé créé: {mapped_df_paris.shape}")
print(f"\n📋 Colonnes finales: {list(mapped_df_paris.columns)}")

✅ DataFrame mappé créé: (12, 19)

��� Colonnes finales: ['id_record', 'titre', 'organisme', 'date_publication', 'date_limite', 'categories', 'montant_max', 'url_source', 'resume', 'public_cible', 'taux_financement', 'contact', 'modalite', 'demarches', 'mots_cles', 'objectif', 'note', 'tags', 'perimetre_geo']


In [ ]:
# Convertir les listes en format Airtable Multiple Select
def convert_pipe_to_list(value):
    """Convertir string avec ||| en liste"""
    if pd.isna(value) or value == '' or value is None:
        return None
    if isinstance(value, str):
        items = [item.strip() for item in value.split('|||') if item.strip()]
        return items if items else None
    if isinstance(value, list):
        return value if value else None
    return None

# Appliquer la conversion
if 'categories' in mapped_df_paris.columns:
    mapped_df_paris['categories'] = mapped_df_paris['categories'].apply(convert_pipe_to_list)
    print("✅ 'categories' converti au format Multiple Select")

if 'mots_cles' in mapped_df_paris.columns:
    mapped_df_paris['mots_cles'] = mapped_df_paris['mots_cles'].apply(convert_pipe_to_list)
    print("✅ 'mots_cles' converti au format Multiple Select")

if 'public_cible' in mapped_df_paris.columns:
    mapped_df_paris['public_cible'] = mapped_df_paris['public_cible'].apply(convert_pipe_to_list)
    print("✅ 'public_cible' converti au format Multiple Select")

✅ 'categories' converti au format Multiple Select
✅ 'mots_cles' converti au format Multiple Select
✅ 'public_cible' converti au format Multiple Select


## 6. Aperçu des données

In [ ]:
print("📊 Résumé des données scrapées:")
print(f"   Total: {len(mapped_df_paris)} enregistrements")
print(f"   Colonnes: {len(mapped_df_paris.columns)}")
print(f"\n📋 Premiers enregistrements:")
mapped_df_paris.head()

��� Résumé des données scrapées:
   Total: 12 enregistrements
   Colonnes: 19

��� Premiers enregistrements:


,id_record,titre,organisme,date_publication,date_limite,categories,montant_max,url_source,resume,public_cible,taux_financement,contact,modalite,demarches,mots_cles,objectif,note,tags,perimetre_geo
0,paris_20260102130659_7133,Les appels à projets de la Ville de Paris,Mairie de Paris,None,None,None,None,https://www.paris.fr/pages/repondre-a-un-appel...,None,None,None,None,None,None,None,None,None,None,Paris
1,paris_20260102130659_7444,Appel à projets pour le partage de la culture ...,Mairie de Paris,2025-01-16,2025-03-10,None,None,https://www.paris.fr/pages/appel-a-projets-202...,None,None,None,None,None,None,None,None,None,None,Paris
2,paris_20260102130659_3733,Appel à projets « Prévenir les conduites à ris...,Mairie de Paris,2025-12-16,2026-02-16,None,None,https://www.paris.fr/pages/appel-a-projets-pre...,None,None,None,None,None,None,None,None,None,None,Paris
3,paris_20260102130659_6413,Appel à projets « Paris Fabrik » : formations ...,Mairie de Paris,2025-12-15,2026-02-19,None,None,https://www.paris.fr/pages/appel-a-projets-par...,None,None,None,None,None,None,None,None,None,None,Paris
4,paris_20260102130659_1238,Appel à projets immobiliers 2026 pour des lieu...,Mairie de Paris,2026-12-10,2026-04-02,None,None,https://www.paris.fr/pages/appel-a-projets-imm...,None,None,None,None,None,None,None,None,None,None,Paris


In [ ]:
# Statistiques
print("📊 Statistiques des données:")
print(f"\n✅ Titres remplis: {mapped_df_paris['titre'].notna().sum()} / {len(mapped_df_paris)}")
print(f"✅ Organismes remplis: {mapped_df_paris['organisme'].notna().sum()} / {len(mapped_df_paris)}")
print(f"✅ Dates limites remplies: {mapped_df_paris['date_limite'].notna().sum()} / {len(mapped_df_paris)}")
print(f"✅ URLs sources remplies: {mapped_df_paris['url_source'].notna().sum()} / {len(mapped_df_paris)}")
print(f"✅ Montants extraits: {mapped_df_paris['montant_max'].notna().sum()} / {len(mapped_df_paris)}")
print(f"✅ Catégories remplies: {mapped_df_paris['categories'].notna().sum()} / {len(mapped_df_paris)}")

��� Statistiques des données:

✅ Titres remplis: 12 / 12
✅ Organismes remplis: 12 / 12
✅ Dates limites remplies: 9 / 12
✅ URLs sources remplies: 12 / 12
✅ Montants extraits: 0 / 12
✅ Catégories remplies: 0 / 12


## 7. Créer fingerprints pour dédupplication

In [ ]:
# Pour fusionner avec d'autres sources, créer une déduplication par fingerprint
def create_fingerprint(row):
    """Créer un fingerprint pour dédupliquer"""
    titre = str(row['titre']) if pd.notna(row['titre']) else ''
    organisme = str(row['organisme']) if pd.notna(row['organisme']) else ''
    date_limite = str(row['date_limite']) if pd.notna(row['date_limite']) else ''
    
    combined = f"{titre}|{organisme}|{date_limite}"
    import hashlib
    return hashlib.md5(combined.encode()).hexdigest()[:12]

mapped_df_paris['fingerprint'] = mapped_df_paris.apply(create_fingerprint, axis=1)
print(f"✅ Fingerprints créés pour dédupplication")

## 8. Exporter les données

In [ ]:
# Exporter en CSV
csv_output = '../data/paris_aap_scraped.csv'
try:
    mapped_df_paris.to_csv(csv_output, index=False, encoding='utf-8')
    print(f"✅ Exporté en CSV: {csv_output}")
except Exception as e:
    print(f"⚠️ Erreur export CSV: {str(e)}")

In [ ]:
# Exporter en JSON
json_output = '../data/paris_aap_scraped.json'
try:
    # Convertir les listes et dates en JSON-compatible
    df_for_json = mapped_df_paris.copy()
    for col in df_for_json.columns:
        if df_for_json[col].dtype == 'object':
            df_for_json[col] = df_for_json[col].astype(str)
    
    df_for_json.to_json(json_output, orient='records', force_ascii=False, indent=2)
    print(f"✅ Exporté en JSON: {json_output}")
except Exception as e:
    print(f"⚠️ Erreur export JSON: {str(e)}")

## 9. Upload vers Airtable (optionnel)

In [ ]:
⚠️ Décommentez pour uploader vers Airtable (nécessite `.env` avec credentials)

In [ ]:
# Décommentez pour uploader
# import sys
# sys.path.append('..')
# from connectors.airtable_connector import AirtableConnector
# 
# airtable = AirtableConnector()
# uploaded = airtable.upload_dataframe(mapped_df_paris)
# print(f"✅ Uploaded {uploaded} records to Airtable!")

print("⏸️  Upload Airtable désactivé (à activer si nécessaire)")